# Ball Screw Drive Surface Defect Dataset for Classification

## 1. Importing Required Libraries

In [1]:
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras.callbacks import History 

## 2. Importing Image of `N` and `P` Class

In [2]:
path1=r"D:\Github\Ball Bearing Fault Detection\Data\testing_data"
cate=["N","P"]

In [3]:
image_size=200
input_image=[]
for i in cate:
    folders=os.path.join(path1,i)
    label=cate.index(i)
    for image in os.listdir(folders):
        image_path=os.path.join(folders,image)
        image_array=cv2.imread(image_path)
        image_array=cv2.resize(image_array,(image_size,image_size))
        input_image.append([image_array,label])

In [4]:
len(input_image)

4000

## 3. Randomly Shuffling Image data

In [5]:
np.random.shuffle(input_image)

## 4. Splitting into X and Y variables

In [30]:
X=[]
Y=[]
for X_values,labels in input_image:
    X.append(X_values)
    Y.append(labels)

## 5. Normalising the data by dividing the x test and train by max pixel size ==255

In [7]:
X=np.array(X)
Y=np.array(Y)
X=(X/255)

In [8]:
X_train=X[0:3000]
Y_train=Y[0:3000]
X_test=X[3000::]
Y_test=Y[3000::]

## 6. Applying VGG19 Model for deep learning

In [9]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPool2D,Flatten,Dense,Dropout

In [10]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, Lambda, Flatten, Input
from keras.preprocessing  import image
from keras.models import Sequential
import glob
import keras

In [11]:
from tensorflow.keras.utils import to_categorical

In [12]:
image_size= [200 , 200]

In [13]:
vgg = VGG16(input_shape = image_size +[3], weights = 'imagenet' ,  include_top = False)
vgg.summary()


Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 200, 200, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 200, 200, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 200, 200, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 100, 100, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 100, 100, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 100, 100, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 50, 50, 128)       0     

In [14]:
for layer in vgg.layers:        
    layer.trainable = False


In [15]:
x = Flatten()(vgg.output)


In [16]:
prediction = Dense(2 , activation = 'sigmoid')(x)


In [17]:
model = Model(vgg.input,prediction)


In [18]:
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 200, 200, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 200, 200, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 200, 200, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 100, 100, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 100, 100, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 100, 100, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 50, 50, 128)       0     

In [19]:
model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy',metrics=["accuracy"] )


In [20]:
from keras.utils import to_categorical
#Y = to_categorical(Y)


In [21]:
model.fit(X_train,Y_train,epochs=6)

Epoch 1/6
94/94 [==============================] - 263s 3s/step - loss: 0.2817 - accuracy: 0.8930
Epoch 2/6
94/94 [==============================] - 256s 3s/step - loss: 0.1365 - accuracy: 0.9540
Epoch 3/6
94/94 [==============================] - 253s 3s/step - loss: 0.0933 - accuracy: 0.9687
Epoch 4/6
94/94 [==============================] - 252s 3s/step - loss: 0.0765 - accuracy: 0.9790
Epoch 5/6
94/94 [==============================] - 252s 3s/step - loss: 0.0605 - accuracy: 0.9843
Epoch 6/6
94/94 [==============================] - 255s 3s/step - loss: 0.0465 - accuracy: 0.9887


## 7. Model Prediction

In [22]:
pred_value=model.predict(X_test)
pred_value      #it gives prediction in probabilites for 2 classes for test data

32/32 [==============================] - 91s 3s/step


array([[2.20718281e-03, 9.96994257e-01],
       [9.76034641e-01, 1.66835394e-02],
       [9.54954326e-01, 9.65693220e-02],
       ...,
       [1.11728255e-02, 9.90088582e-01],
       [7.03609476e-05, 9.99927163e-01],
       [5.00579514e-02, 9.60115850e-01]], dtype=float32)

In [23]:
pred_classes=np.argmax(pred_value,axis=1)

## 8. Result

In [24]:
from sklearn.metrics import confusion_matrix,accuracy_score,recall_score,f1_score,precision_score

In [25]:
tab=confusion_matrix(Y_test,pred_classes)
tab

array([[494,  15],
       [ 13, 478]], dtype=int64)

In [26]:
accuracy=accuracy_score(Y_test,pred_classes)*100
accuracy

97.2

In [27]:
precision=precision_score(Y_test,pred_classes)*100
precision

96.95740365111561

In [28]:
recall=recall_score(Y_test,pred_classes)*100
recall

97.35234215885947

In [29]:
f1_Score=f1_score(Y_test,pred_classes)*100
f1_Score

97.15447154471545